In [1]:
import sys
import xgboost as xgb
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from datetime import datetime
from itertools import combinations

In [2]:
#APERTURA DE ARCHIVO DE ARCHIVOS
entrenamiento = pd.read_csv("entrenamiento-listo.csv")
test = pd.read_csv("test-listo.csv")

In [3]:
#FILTRADO DE COLUMNAS - NO REMOVER STAGE O FECHA
entrenamiento = entrenamiento.drop(columns=['ID','Opportunity_Name','Sales_Contract_No'])
test = test.drop(columns=['ID','Opportunity_Name','Sales_Contract_No'])

In [4]:
entrenamiento = entrenamiento.replace([np.inf, -np.inf], np.nan).dropna()
entrenamiento_label = (entrenamiento['Stage'] == 'Closed Won').astype(int)
entrenamiento = entrenamiento.drop(columns=['Stage','Fecha'])

test = test.replace([np.inf, -np.inf], np.nan).dropna()
test_label          = (test['Stage'] == 'Closed Won').astype(int)
test = test.drop(columns=['Stage','Fecha'])

scaler = StandardScaler()
scaler.fit(entrenamiento)
entrenamiento = scaler.transform(entrenamiento)
test = scaler.transform(test)

classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(entrenamiento, entrenamiento_label)
y_pred = classifier.predict(test)

print(confusion_matrix(test_label, y_pred))
print(classification_report(test_label, y_pred))

classifier.fit(entrenamiento, entrenamiento_label)
y_pred = classifier.predict(entrenamiento)
print(confusion_matrix(entrenamiento_label, y_pred))
print(classification_report(entrenamiento_label, y_pred))

[[765 159]
 [673 680]]
              precision    recall  f1-score   support

           0       0.53      0.83      0.65       924
           1       0.81      0.50      0.62      1353

    accuracy                           0.63      2277
   macro avg       0.67      0.67      0.63      2277
weighted avg       0.70      0.63      0.63      2277

[[4490  494]
 [ 382 6184]]
              precision    recall  f1-score   support

           0       0.92      0.90      0.91      4984
           1       0.93      0.94      0.93      6566

    accuracy                           0.92     11550
   macro avg       0.92      0.92      0.92     11550
weighted avg       0.92      0.92      0.92     11550

